**This notebook is to evaluate the generalizability of the CNN-driven advection scheme.**

We computed the statistical metrics with the solver in an epoch to see if the learned model can work well on the new dataset.
This notebook is specifically for evaluating the robustness of solver against the changed domain direction (longitudinal).

In [1]:
## Import packages
#import Pkg
#Pkg.add("Plots")
using Plots
#Pkg.add("BenchmarkTools")
using BenchmarkTools
#Pkg.add("Flux")
using Flux
#Pkg.add("CSV")
using CSV
#Pkg.add("DelimitedFiles")
using DelimitedFiles
#Pkg.add("Statistics")
using Statistics
#Pkg.add("StatsBase")
using StatsBase
#Pkg.add("BSON")
using BSON: @save
using BSON: @load
#Pkg.add("Random")
using Random

In [2]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 3 stencil (default) ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim_total, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    s1_bc = zeros(xdim_total+2)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 3))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[i] for i in 1:xdim_total], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2]), xdim_total, 3)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [107]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 5 stencil ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    s1_bc = zeros(xdim_total+4)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 5))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[1]], [su[i] for i in 1:xdim_total], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4]), xdim_total, 5)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [122]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 9 stencil ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    #s1_bc = zeros(xdim_total+4)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 9))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
            [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8]), xdim_total, 9)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [37]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 17 stencil ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    #s1_bc = zeros(xdim_total+4)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 17))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
            [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], 
            [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8],
                [s1_bc[i] for i in 10:xdim_total+9],
                [s1_bc[i] for i in 11:xdim_total+10],
                [s1_bc[i] for i in 12:xdim_total+11],
                [s1_bc[i] for i in 13:xdim_total+12],
                [s1_bc[i] for i in 14:xdim_total+13],
                [s1_bc[i] for i in 15:xdim_total+14],
                [s1_bc[i] for i in 16:xdim_total+15],
                [s1_bc[i] for i in 17:xdim_total+16]), xdim_total, 17)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [39]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 31 stencil ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    #s1_bc = zeros(xdim_total+4)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 31))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], 
                [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]],
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]],
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8],
                [s1_bc[i] for i in 10:xdim_total+9],
                [s1_bc[i] for i in 11:xdim_total+10],
                [s1_bc[i] for i in 12:xdim_total+11],
                [s1_bc[i] for i in 13:xdim_total+12],
                [s1_bc[i] for i in 14:xdim_total+13],
                [s1_bc[i] for i in 15:xdim_total+14],
                [s1_bc[i] for i in 16:xdim_total+15],
                [s1_bc[i] for i in 17:xdim_total+16],
                [s1_bc[i] for i in 18:xdim_total+17],
                [s1_bc[i] for i in 19:xdim_total+18],
                [s1_bc[i] for i in 20:xdim_total+19],
                [s1_bc[i] for i in 21:xdim_total+20],
                [s1_bc[i] for i in 22:xdim_total+21],
                [s1_bc[i] for i in 23:xdim_total+22],
                [s1_bc[i] for i in 24:xdim_total+23],
                [s1_bc[i] for i in 25:xdim_total+24],
                [s1_bc[i] for i in 26:xdim_total+25],
                [s1_bc[i] for i in 27:xdim_total+26],
                [s1_bc[i] for i in 28:xdim_total+27],
                [s1_bc[i] for i in 29:xdim_total+28],
                [s1_bc[i] for i in 30:xdim_total+29],
                [s1_bc[i] for i in 31:xdim_total+30]), xdim_total, 31)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [38]:
## Select the spatial and temporal coarsening factors to be evaluated
## Specify the training epoch as well. At first, you could try the optimal epoch in training and see the results.
## If the optimal epoch in training did not give a model converging, you could try different epoch.

space_factor = 1
time_factor = 32
EPOCHS = 225
if space_factor < 10
    space_name = string(0)*string(space_factor)
else
    space_name = string(space_factor)
end
if time_factor < 10
    time_name = string(0)*string(time_factor)
else
    time_name = string(time_factor)
end

"32"

In [39]:
## This cell is used to test the trained model with different scenario.

vel_GEOS_Array = readdlm("Generalization_tests/Longitudinal/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_"*string(space_factor)*"x_"*string(time_factor)*"x.csv", ',', Float32);
scalar_GEOS_Array = readdlm("Generalization_tests/Longitudinal/VL_GEOS_Jan_2019_NASA_GMAO_10_U_"*string(space_factor)*"x_"*string(time_factor)*"x.csv", ',', Float32);
scalar_coarse_numerical = readdlm("Generalization_tests/Longitudinal/VanLeer_Coarse_GEOS_Jan_2019_NASA_GMAO_10_U_"*string(space_factor)*"x_"*string(time_factor)*"x.csv", ',', Float32);

xdim = size(vel_GEOS_Array, 1);
nstep = size(vel_GEOS_Array, 2);

Random.seed!(1)
history = (scalar_GEOS_Array*Float32(1e7), vel_GEOS_Array/15);

## Call variables
coeff_estimated = zeros(Float32, xdim,2)
#input = coarse_grained_s_2x[:,:]
s2_2x = convert(Array{Float64}, zeros(xdim))
history_2x_learned = zeros(Float32, xdim, 1, nstep);

input_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
u_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
target_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)

input_NN_integrate[:,1,1,:] = history[1][:,1:nstep-1]
u_NN_integrate[:,1,1,:] = history[2][:,1:nstep-1]
target_NN_integrate[:,1,1,:] = history[1][:,2:nstep]

#@load "Train_outputs/"*string(space_name)*"x"*string(time_name)*"t/CNN_DLR_"*string(EPOCHS)*"EPOCHS_"*string(space_name)*"X"*string(time_name)*"X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
#@load "Train_outputs/"*string(space_name)*"x"*string(time_name)*"t_2/CNN_DLR_"*string(EPOCHS)*"EPOCHS_"*string(space_name)*"X"*string(time_name)*"X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
@load "Train_outputs/"*string(space_name)*"x"*string(time_name)*"t_4layer/CNN_DLR_"*string(EPOCHS)*"EPOCHS_"*string(space_name)*"X"*string(time_name)*"X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
#@load "Train_outputs/"*string(space_name)*"x"*string(time_name)*"t/CNN_DLR_"*string(EPOCHS)*"EPOCHS_1X1X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
#@load "Train_outputs/"*string(space_name)*"x"*string(time_name)*"t/CNN_DLR_"*string(EPOCHS)*"EPOCHS_1X2X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
ps = Flux.params(model);

In [40]:
## Model run
# 29N dx = 30276.7
# 39N dx = 27034.3
# 45N dx = 24597.8
# Longitudinal dy = 27829.3
@time CNN_scalar = Float32(1e-7)*pgm_ml(input_NN_integrate, u_NN_integrate, model, xdim, nstep, Float32(27829.3*space_factor), Float32(300*time_factor))[:,1,:];

  2.321902 seconds (1.83 M allocations: 173.209 MiB, 1.03% gc time, 26.19% compilation time)


In [41]:
## Calculate error metrics
dim = xdim*nstep

mae_learned = StatsBase.L1dist(scalar_GEOS_Array, CNN_scalar)/(dim)
rmse_learned = StatsBase.L2dist(scalar_GEOS_Array, CNN_scalar)/sqrt(dim)
r2_learned = Statistics.cor(reshape(scalar_GEOS_Array, dim), reshape(CNN_scalar, dim))^2

mae_numerical = StatsBase.L1dist(scalar_GEOS_Array, scalar_coarse_numerical)/(dim)
rmse_numerical = StatsBase.L2dist(scalar_GEOS_Array, scalar_coarse_numerical)/sqrt(dim)
r2_numerical = Statistics.cor(reshape(scalar_GEOS_Array, dim), reshape(scalar_coarse_numerical, dim))^2
    
println("results: ", "   ", mae_learned, "   ", rmse_learned, "   ", r2_learned)
println("results: ", "   ", mae_numerical, "   ", rmse_numerical, "   ", r2_numerical)

results:    1.344564593593181e-8   2.633898945750583e-8   0.7084246
results:    1.4165139911411084e-6   3.8580217395051464e-5   0.00020694386
